# AutoGen Tool Use
LLM이 사용자의 지시에 따라 임의의 코드를 작성하는 것은 매우 쉽지만, 어떤 코드를 작성하게 할지 제어하는 건 어려울 수 있음.

이걸 위해서 Tool Use를 사용한다~!

Tool은 에이전트가 사용할 수 있도록 미리 정의된 함수이다. 

에이전트는 임의의 코드를 작성하는 대신, 도구를 호출하여 웹 검색, 계산 수행, 파일 읽기 또는 원격 API 호출과 같은 작업을 수행할 수 있다. 에이전트가 사용할 수 있는 Tool을 제어함으로써, 에이전트가 수행할 수 있는 작업들을 통제할 수 있다.

> **참고**  
> Tool Use는 현재 OpenAI와 호환되는 tool call API를 지원하는 LLM에서만 사용할 수 있음.

## Creating Tools

Tool은 일반적인 Python 함수로 생성할 수 있다.

한 번에 하나의 연산만 수행할 수 있는 계산기를 만들어보자~


Tool을 정의할 때, Type Hint를 사용하면 더 잘 알아먹는다고 하네요~ (역시 지훈갓)

In [1]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

이렇게 Tool을 정의하면, 에이전트에 Tool을 등록해야 함

`register_for_llm`: Tool을 호출하기 위해 등록

`register_for_execution`: Tool을 실행하기 위해 등록

In [2]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

아래처럼 등록할 수도 있음

In [3]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

/root/miniconda3/envs/autogen/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:2955: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/root/miniconda3/envs/autogen/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:2875: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


In [4]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?")

User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_PNYkXrxzxAxyhdRSKWdTET9R): calculator *****
Arguments: 
{"a":232,"b":32,"operator":"-"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: call_PNYkXrxzxAxyhdRSKWdTET9R
Input arguments: {'a': 232, 'b': 32, 'operator': '-'}
User (to Assistant):

***** Response from calling tool (call_PNYkXrxzxAxyhdRSKWdTET9R) *****
200
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_6vKF1DLR26KxvptM4iVBOn9z): calculator *****
Arguments:

In [5]:
(44232 + int(13312 / (232 - 32))) * 5

221490

Tool을 정의할 때, 어떻게 파이썬 함수를 전달해주는 것만으로 저것을 LLM이 알아먹을 수 있던 것일까?

OPEN AI Tool Call API가 저절로 스키마 생성함.

In [6]:
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A simple calculator',
   'name': 'calculator',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer', 'description': 'a'},
     'b': {'type': 'integer', 'description': 'b'},
     'operator': {'enum': ['+', '-', '*', '/'],
      'type': 'string',
      'description': 'operator'}},
    'required': ['a', 'b', 'operator']}}}]

더 정확하게 Tool 정의를 하고 싶다면 Pydantic을 사용하면 된다~

In [7]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: Annotated[int, Field(description="The first number.")]
    b: Annotated[int, Field(description="The second number.")]
    operator: Annotated[Operator, Field(description="The operator.")]


def calculator(input: Annotated[CalculatorInput, "Input to the calculator."]) -> int:
    if input.operator == "+":
        return input.a + input.b
    elif input.operator == "-":
        return input.a - input.b
    elif input.operator == "*":
        return input.a * input.b
    elif input.operator == "/":
        return int(input.a / input.b)
    else:
        raise ValueError("Invalid operator")

In [8]:
assistant.register_for_llm(name="calculator", description="A calculator tool that accepts nested expression as input")(
    calculator
)
user_proxy.register_for_execution(name="calculator")(calculator)

In [9]:
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A calculator tool that accepts nested expression as input',
   'name': 'calculator',
   'parameters': {'type': 'object',
    'properties': {'input': {'properties': {'a': {'description': 'The first number.',
        'title': 'A',
        'type': 'integer'},
       'b': {'description': 'The second number.',
        'title': 'B',
        'type': 'integer'},
       'operator': {'description': 'The operator.',
        'enum': ['+', '-', '*', '/'],
        'title': 'Operator',
        'type': 'string'}},
      'required': ['a', 'b', 'operator'],
      'title': 'CalculatorInput',
      'type': 'object',
      'description': 'Input to the calculator.'}},
    'required': ['input']}}}]

In [12]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (1423 - 123) / 3 + (32 + 23) * 5?")

User (to Assistant):

What is (1423 - 123) / 3 + (32 + 23) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_dHriAFm36REE2qsp7Se8K2hb): calculator *****
Arguments: 
{"input":{"a":1423,"b":123,"operator":"-"},"b":3,"operator":"/"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: call_dHriAFm36REE2qsp7Se8K2hb
Input arguments: {'input': {'a': 1423, 'b': 123, 'operator': '-'}, 'b': 3, 'operator': '/'}
User (to Assistant):

***** Response from calling tool (call_dHriAFm36REE2qsp7Se8K2hb) *****
1300
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggeste

In [11]:
int((1423 - 123) / 3) + (32 + 23) * 5


708